## Code to mount the drive, the folder NLP_2 is made and has all the categories with their articles are saved. ⚡

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Code to fetch data from wikipedia, language parameter of the method is to choose the language of the category that is being fetched

In [2]:
import requests
import codecs
import os



def savePage(title,folder, language ='en'):
    base_url = 'https://{}.wikipedia.org/w/api.php'.format(language)
    response = requests.get(
    base_url,
    params={
        'action': 'query',
        'format': 'json',
        'titles': title,
        'prop': 'extracts',
        'explaintext': True,
        'language': 'de'
        }
    ).json()

    text = ''
    pagelist = response.get('query').get('pages')
    for page in pagelist:
        wikipage = pagelist[page]
        text = wikipage['extract']
        break

    if not os.path.isdir(folder):
        os.mkdir(folder)
    f = codecs.open(folder+'/'+title.replace(' ','_')+'.txt','w','utf8')
    f.write(text)
    f.close()
    return


def getWikiPages(category, language= 'en'):
    titles = []
    base_url = 'https://{}.wikipedia.org/w/api.php'.format(language)
    print(base_url)
    response = requests.get(
        base_url,
        params={
            'action': 'query',
            'list': 'categorymembers',
            'cmtitle': 'Category:'+category,
            'cmtype':'page',
            'format': 'json'
        }
    ).json()


    while(response):
        titles.extend( [t['title'] for t in response['query']['categorymembers']] )
        if 'continue' in response:
            cont = response['continue']['cmcontinue']
            response = requests.get(
                base_url,
                params={
                    'action': 'query',
                    'list': 'categorymembers',
                    'cmtitle': 'Category:' + category,
                    'cmtype': 'page',
                    'cmcontinue': cont,
                    'format': 'json'
                }
            ).json()
        else:
            break

        if not cont:
            response = None

    return titles


def downloadWikiCat(category,folder,recursive=False, language= 'en'):
    titles = getWikiPages(category, language)
    for t in titles:
        if r'/' not in t:
           savePage(t,folder, language)
    return

### This piece of code calls the above wikipedia api and store the articles under the category chosen for both english and german articles of the similar category

In [ ]:
# # Histroy , Infectious diseases , Technology

# wiki_category = 'Technology'
# save_folder = '/content/drive/MyDrive/NLP_2/tech_en'

# downloadWikiCat(wiki_category, save_folder, language='en')

In [ ]:
# # Geschichte , Infektionskrankheit , Technologie

# wiki_category = 'Technologie'
# save_folder = '/content/drive/MyDrive/NLP_2/tech_de'

# downloadWikiCat(wiki_category, save_folder, language='de')

In [ ]:
%ls

drive/  sample_data/


In [ ]:
# # this gets teh current folder path
# current_folder = os.getcwd()

# # list of folders to be merged
# list_dir = ['disease_de', ' history_de', 'tech_de',
#             'disease_en', 'history_en','tech_en']
# root_folder = '/content/drive/MyDrive/NLP_2'
# output_file = 'combined_files.txt'

# # Function to combine files in a folder
# def combine_files_in_folder(folder_path):
#     combined_data = []
#     for dirpath, dirnames, filenames in os.walk(folder_path):
#         for filename in filenames:
#             file_path = os.path.join(dirpath, filename)
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 combined_data.append(file.read())
#     return combined_data

# # # Create or open the output file for writing
# # with open(output_file, 'w', encoding='utf-8') as output:
# #     for sub_folder in os.listdir(root_folder):
# #         sub_folder_path = os.path.join(root_folder, sub_folder)
# #         if os.path.isdir(sub_folder_path):
# #             combined_data = combine_files_in_folder(sub_folder_path)
# #             # Write the combined data to the output file
# #             output.writelines(combined_data)
# #             output.write('\n')  # Add a separator between folder content

# # print(f'Combined files from {root_folder} into {output_file}')


In [ ]:
# output_eng_file = '/content/drive/MyDrive/NLP_2/combined_english.txt'
# output_ger_file = '/content/drive/MyDrive/NLP_2/combined_german.txt'

# with open(output_eng_file, 'w', encoding='utf-8') as output:
#   eng_combined = combine_files_in_folder('/content/drive/MyDrive/NLP_2/english')
#   output.writelines(eng_combined)

# with open(output_ger_file, 'w', encoding='utf-8') as output:
#   ger_combined = combine_files_in_folder('/content/drive/MyDrive/NLP_2/german')
#   output.writelines(ger_combined)

## Combining articles of different language got from Wikipedia in one file to preprocess and then make word embeddings

#Shutil - High-level File Operation Library
The shutil module offers a number of high-level operations on files and collections of files. In particular, functions are provided which support file copying and removal.

In [ ]:
import shutil
import glob

folder_path = "/content/driver/MyDrive/NLP_2/german/history_de"

# This part actually lists all files with a .txt extension in the specified folder
txt_files = glob.glob(f'{folder_path}/*.txt')

with open('/content/drive/MyDrive/NLP_2/german_corpus', 'wb') as outfile:
    for filename in txt_files:
        if filename == '/content/drive/MyDrive/NLP_2/german_corpus':
            continue
        with open(filename, 'rb') as readfile:
            shutil.copyfileobj(readfile, outfile)



## Combining the two corpora to perform preprocessing steps and then train the tokens embeddings

In [ ]:
# filenames = ['/content/drive/MyDrive/NLP_2/german_corpus', '/content/drive/MyDrive/NLP_2/english_corpus']
# with open('/content/drive/MyDrive/NLP_2/combined_corpus', 'w') as outfile:
#     for fname in filenames:
#         with open(fname) as infile:
#             for line in infile:
#                 outfile.write(line)

# Pre-processing steps on bilingual corpus

## using text_preprocessing for performing pre-processing steps

In [3]:
!pip install text_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.1 MB/s eta 0:00:00


In [4]:
from text_preprocessing import preprocess_text
from text_preprocessing import to_lower, remove_email, remove_url, remove_punctuation, lemmatize_word, keep_alpha_numeric,tokenize_word, remove_stopword

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


### here I am first preprocessing both teh english and german corpus and then making token of the processed text

In [9]:
with open('/content/drive/MyDrive/NLP_2/combined_german.txt') as file:
    sentences_file_german = file.read()

with open('/content/drive/MyDrive/NLP_2/combined_english.txt') as file:
    sentences_file_english = file.read()

with open('/content/drive/MyDrive/NLP_2/combined_files.txt') as file:
    sentences_file_combined = file.read()
#print(sentences_file)
# Preprocess text using custom preprocess functions in the pipeline
preprocess_functions = [remove_url, remove_stopword]
preprocessed_text_german = preprocess_text(sentences_file_german, preprocess_functions)
preprocessed_text_english = preprocess_text(sentences_file_english, preprocess_functions)
preprocessed_text_combined = preprocess_text(sentences_file_combined, preprocess_functions)

## making tokens of the processed text

In [10]:
import nltk
from nltk.tokenize import (word_tokenize, sent_tokenize, TreebankWordDetokenizer, wordpunct_tokenize, TweetTokenizer, MWETokenizer)

word_tokenized_english = []
word_tokenized_german = []
word_token_combine = []
word_tokenized_english.append(wordpunct_tokenize(preprocessed_text_english))
word_tokenized_german.append(wordpunct_tokenize(preprocessed_text_german))
word_token_combine.append(wordpunct_tokenize(preprocessed_text_combined))

## These german and english token are then passed in the gensim Word2Vec model for generating embeddings with skip-gram model and they are saved individually

In [24]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Train the Word2Vec model for bilingual embeddings
model_german = Word2Vec(sentences=word_tokenized_german, vector_size=200, window=7, min_count=1, sg=1)
model_english = Word2Vec(sentences=word_tokenized_english, vector_size=200, window=7, min_count=1, sg=1)
model_combine = Word2Vec(sentences=word_token_combine, vector_size=200, window=7, min_count=1, sg=1)
# Saving the trained embeddings
model_german.save("german_word2vec.model")
model_english.save("english_word2vec.model")
model_combine.save("combined_word2vec.model")
# Storing just the words + their trained embeddings.
#german word vector
wv_german = model_german.wv
wv_german.save("german_word2vec.wordvectors")

# english word vector
wv_english = model_english.wv
wv_english.save("english_word2vec.wordvectors")


In [25]:
# english word vector
wv_combine = model_combine.wv
wv_combine.save("combine_word2vec.wordvectors")

Now these embeddings needs to be tested in terms of their similarity and relations/ context(translation) in both languages, so Average Precision of word pair(english word, german translation) is listed and evaluated

In [12]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics import average_precision_score
from scipy.spatial.distance import cosine

## Defining a list of words for evaluation (English and their translations in German)
# word_pairs = [("disease", "Krankheit"), ("Mutations", "Mutationen"), ("used", "eingesetzt")]

word_pairs = [("disease", "Krankheit"), ("dates", "Daten"), ("used", "eingesetzt"), ("Mexico", "Mexiko")]
ap_scores = []

# Calculate Average Precision (AP) for each word pair
for english_word, german_word in word_pairs:
    try:
        # Find the most similar terms from L1 (English) for B (German)
        similar_terms_from_l1 = [word[0] for word in model_english.wv.most_similar(positive=[model_german.wv[german_word]])]
        print(similar_terms_from_l1)

        # Calculating AP based on the position of A (English word) in the list
        ap = 0.0
        num_correct = 0

        for i, term in enumerate(similar_terms_from_l1, 1):
          term_vector = model_english.wv[term]
          eng_vector = model_english.wv[english_word]
          cosine_sim = cosine(eng_vector, term_vector)
          if cosine_sim >= 0.9:
                num_correct += 1
                ap += num_correct / i

        if num_correct > 0:
            ap /= num_correct

        ap_scores.append(ap)

        print(f"Word Pair: ({english_word}, {german_word}), AP: {ap:.2f}")
    except KeyError:
        print(f"One or both of the words not found in vocabulary.")

# Calculate Mean Average Precision (MAP)
map_score = np.mean(ap_scores)

print(f"Mean Average Precision (MAP): {map_score:.2f}")

['Toynbee', 'intervals', 'exposed', 'Furet', 'classics', 'Fasolt', 'really', 'instant', 'classificatory', 'chapters']
Word Pair: (disease, Krankheit), AP: 1.00
['Toynbee', 'intervals', 'exposed', 'Furet', 'classics', 'really', 'instant', 'Fasolt', 'separates', 'untaught']
Word Pair: (dates, Daten), AP: 0.00
['Toynbee', 'exposed', 'intervals', 'Fasolt', 'separates', 'Furet', 'classics', 'really', 'classificatory', 'instant']
Word Pair: (used, eingesetzt), AP: 0.00
['Toynbee', 'exposed', 'intervals', 'Fasolt', 'Furet', 'classificatory', 'classics', 'separates', 'floruit', 'chapters']
Word Pair: (Mexico, Mexiko), AP: 0.00
Mean Average Precision (MAP): 0.25


In [22]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics import average_precision_score
from scipy.spatial.distance import cosine

## Defining a list of words for evaluation (English and their translations in German)
# word_pairs = [("disease", "Krankheit"), ("Mutations", "Mutationen"), ("used", "eingesetzt")]

word_pairs = [("disease", "Krankheit"),
    ("dates", "Daten"),
    ("used", "eingesetzt"),
    ("Mexico", "Mexiko"),
    ("infection", "Infektion"),
    ("treatment", "Behandlung"),
    ("vaccine", "Impfstoff"),
    ("outbreak", "Ausbruch"),
    ("symptoms", "Symptome"),
    ("immune", "immun"),
    ("diagnosis", "Diagnose"),
    ("prevention", "Prävention"),
    ("hospital", "Krankenhaus")]
ap_scores = []

# Calculate Average Precision (AP) for each word pair
for english_word, german_word in word_pairs:
    try:
        # Find the most similar terms from L1 (English) for B (German)
        similar_terms_from_l1 = [word[0] for word in model_combine.wv.most_similar(positive=[model_combine.wv[german_word]])]
        print(similar_terms_from_l1)

        # Calculating AP based on the position of A (English word) in the list
        ap = 0.0
        num_correct = 0

        for i, term in enumerate(similar_terms_from_l1, 1):
          if term == german_word:
            num_correct += 1
            ap += num_correct / i

        if num_correct > 0:
            ap /= num_correct

        ap_scores.append(ap)

        print(f"Word Pair: ({english_word}, {german_word}), AP: {ap:.2f}")
    except KeyError:
        print(f"One or both of the words not found in vocabulary.")

# Calculate Mean Average Precision (MAP)
map_score = np.mean(ap_scores)

print(f"Mean Average Precision (MAP): {map_score:.2f}")

['Krankheit', 'Marianne', 'Pflanzenteilen', 'wölbt', 'Metapneumovirus', 'Zerebrospinalflüssigkeit', 'GmbH', 'absondern', 'evoking', 'Barrett']
Word Pair: (disease, Krankheit), AP: 1.00
['Daten', 'Austro', 'weapon', 'Marzahn', 'unwanted', 'tolerize', 'expounded', 'schwerere', 'exogenen', 'protist']
Word Pair: (dates, Daten), AP: 1.00
['eingesetzt', 'isoniazid', 'Bohrdrähte', 'verminderten', 'Democritus', '4196', 'spacecraft', 'Könige', 'defecations', 'controversia']
Word Pair: (used, eingesetzt), AP: 1.00
['Mexiko', 'justification', 'Ornithology', 'Merwin', 'Schiebold', 'Gaimard', 'swollen', 'Concordiae', 'Peck', 'Sehnenscheidenentzündung']
Word Pair: (Mexico, Mexiko), AP: 1.00
['Infektion', 'Verbrennungsmaschine', 'Vorfall', 'Neurologische', 'Naison', 'odontogen', 'Frühsymptome', 'Ausbrüche', 'Nebenhodens', 'proceeded']
Word Pair: (infection, Infektion), AP: 1.00
['Behandlung', 'nachweisen', 'Action', 'Browser', 'maltophilia', 'knowing', 'hämatogenen', 'Wandertaubenembryo', 'entails', 

trying teh above code with more word pairs,

In [ ]:
word_pairs = [
    ("disease", "Krankheit"),
    ("Mexico", "Mexiko"),
    ("used", "eingesetzt"),
    ("infection", "Infektion"),
    ("treatment", "Behandlung"),
    ("vaccine", "Impfstoff"),
    ("outbreak", "Ausbruch"),
    ("symptoms", "Symptome"),
    ("immune", "immun"),
    ("diagnosis", "Diagnose"),
    ("prevention", "Prävention"),
    ("hospital", "Krankenhaus")
]

for english_word, german_word in word_pairs:
    try:
        similar_terms_from_l1 = [word[0] for word in model_english.wv.most_similar(positive=[model_german.wv[german_word]])]
        ap = 0.0
        num_correct = 0

        for i, term in enumerate(similar_terms_from_l1, 1):
          term_vector = model_english.wv[term]
          eng_vector = model_english.wv[english_word]
          cosine_sim = cosine(eng_vector, term_vector)
          if cosine_sim >= 0.9:
                num_correct += 1
                ap += num_correct / i

        if num_correct > 0:
          ap /= num_correct
        ap_scores.append(ap)
        print(f"Word Pair: ({english_word}, {german_word}), AP: {ap:.2f}")
    except KeyError:
        print(f"One or both of the words not found in vocabulary.")

# Calculate Mean Average Precision (MAP)
map_score = np.mean(ap_scores)

print(f"Mean Average Precision (MAP): {map_score:.2f}")


Word Pair: (disease, Krankheit), AP: 1.00
Word Pair: (Mexico, Mexiko), AP: 0.00
Word Pair: (used, eingesetzt), AP: 0.00
Word Pair: (infection, Infektion), AP: 1.00
Word Pair: (treatment, Behandlung), AP: 0.64
Word Pair: (vaccine, Impfstoff), AP: 1.00
Word Pair: (outbreak, Ausbruch), AP: 0.00
Word Pair: (symptoms, Symptome), AP: 0.90
Word Pair: (immune, immun), AP: 0.96
Word Pair: (diagnosis, Diagnose), AP: 0.99
Word Pair: (prevention, Prävention), AP: 0.88
Word Pair: (hospital, Krankenhaus), AP: 0.47
Mean Average Precision (MAP): 0.64


## In the beginning of my implementation I didn't get any good average precisions, assuming the embeddings are not good enough with the basic token. Here using BPE, Word Piece algorithm for tokenization to better capture the morphological essence in german text. trying above task with new tokenization method and embeddings

In [ ]:
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from tokenizers import Tokenizer
from gensim.parsing.preprocessing import STOPWORDS
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece
from tokenizers.trainers import BpeTrainer, WordLevelTrainer, \
                                WordPieceTrainer, UnigramTrainer

## a pretokenizer to segment the text into words
from tokenizers.pre_tokenizers import Whitespace
from pprint import pprint

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
unk_token = "<UNK>"  # token for unknown words
spl_tokens = ["<UNK>", "<SEP>", "<MASK>", "<CLS>"]  # special tokens

def prepare_tokenizer_trainer(alg):
    """
    Prepares the tokenizer and trainer with unknown & special tokens.
    """
    if alg == 'BPE':
        tokenizer = Tokenizer(BPE(unk_token = unk_token))
        trainer = BpeTrainer(special_tokens = spl_tokens)
    elif alg == 'WPC':
        tokenizer = Tokenizer(WordPiece(unk_token = unk_token))
        trainer = WordPieceTrainer(special_tokens = spl_tokens)
    else:
        tokenizer = Tokenizer(WordLevel(unk_token = unk_token))
        trainer = WordLevelTrainer(special_tokens = spl_tokens)


    tokenizer.pre_tokenizer = Whitespace()
    tokenizer.stopwords = STOPWORDS
    return tokenizer, trainer

def tokenize(input_string, tokenizer):
    """
    Tokenizes the input string using the tokenizer provided.
    """
    output = tokenizer.encode(input_string)
    return output

In [ ]:
def train_tokenizer(files, alg='BPE'):
    """
    Takes the files and trains the tokenizer.
    """
    tokenizer, trainer = prepare_tokenizer_trainer(alg)
    tokenizer.train(files, trainer) # training the tokenzier
    tokenizer.save("./tokenizer-trained.json")
    tokenizer = Tokenizer.from_file("./tokenizer-trained.json")
    return tokenizer

For creating tokens, I have combined english and german text in a single file and then performed preprocessing and used above mentioned algorithms for token creation. Also the same is suggested in the task by the professor|

In [ ]:
with open('/content/drive/MyDrive/NLP_2/combined_files.txt') as file:
    sentences_file = file.read().splitlines()

preprocessed_text =[]
tokenized_sentences = []
trained_tokenizer = train_tokenizer(['/content/drive/MyDrive/NLP_2/combined_files.txt'], 'WPC')


In [ ]:
with open('/content/drive/MyDrive/NLP_2/combined_files.txt') as file:
    sentences_file = file.read()


preprocessed_text =[]
tokenized_sentences = []
preprocessed_text =[]

# change the variable name to combine
preprocess_functions = [remove_punctuation, remove_url, remove_stopword]
preprocessed_text_german = preprocess_text(sentences_file, preprocess_functions)
encoding_ger = tokenize(preprocessed_text_german,trained_tokenizer)
subword_tokens = encoding_ger.tokens
tokenized_sentences.append(subword_tokens)



In [ ]:
model = Word2Vec(sentences=tokenized_sentences, vector_size=200, window=7, min_count=1, sg=1)

# Saving the trained model combined
model.save("combined_word2vec.model")



In [35]:
import numpy as np
from scipy.spatial.distance import cosine

word_pairs = [
    ("disease", "Krankheit"),
    ("Mexico", "Mexiko"),
    ("Bronze", "Bronze"),
    ("humanity", "Menschheit"),
    ("cook", "Kochen")
]

ap_scores = []
dict_1 ={}

for english_word, german_word in word_pairs:
    try:
        similar_terms_from_l1 = [word[0] for word in model.wv.most_similar(negative=[model.wv[german_word]])]
        print(similar_terms_from_l1)
        ap = 0.0
        num_correct = 0

        for i, term in enumerate(similar_terms_from_l1, 1):
          if term == german_word:
                num_correct += 1
                ap += num_correct / i

        if num_correct > 0:
          ap /= num_correct

        ap_scores.append(ap)

        print(f"Word Pair: ({english_word}, {german_word}), AP: {ap:.2f}")
    except KeyError:
        print(f"One or both of the words not found in vocabulary.")

map_score = np.mean(ap_scores)
print(f"Mean Average Precision (MAP): {map_score:.2f}")


['1869', 'Wurzel', 'Nuon', 'yearly', 'termed', 'modes', '##asen', '##事', 'Camer', 'Chlorella']
Word Pair: (disease, Krankheit), AP: 0.82
One or both of the words not found in vocabulary.
['##9', 'propagate', '##authorized', 'acetylcholine', 'Leistungen', '##bungen', 'festiv', 'pathologies', 'lateinische', 'sant']
Word Pair: (Bronze, Bronze), AP: 1.00
['Komp', '##führer', 'drainage', 'geeigneten', '##gegeben', 'CD4', 'Modern', 'wrest', 'Allgemeinen', 'Mundhöhle']
Word Pair: (humanity, Menschheit), AP: 0.99
['##aea', '##ubles', 'neben', 'Off', 'attendance', 'helfen', 'pancreatic', 'peel', 'Handbuch', 'viele']
Word Pair: (cook, Kochen), AP: 0.58
Mean Average Precision (MAP): 0.85


# from here on, these code lines are for understanding the data and extracting data statistics

The below code calculates the total word count in both english and german text files



In [36]:
import os

def count_words_in_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            words = content.split()
            return len(words)
    except FileNotFoundError:
        return 0

def count_words_in_folder(folder_path):
    folder_word_count = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.txt'):
                file_path = os.path.join(dirpath, filename)
                folder_word_count += count_words_in_file(file_path)
    return folder_word_count

def count_words_in_folders(root_folder):
    total_word_count = 0
    for sub_folder in os.listdir(root_folder):
        sub_folder_path = os.path.join(root_folder, sub_folder)
        if os.path.isdir(sub_folder_path):
            folder_word_count = count_words_in_folder(sub_folder_path)
            total_word_count += folder_word_count
            print(f"Folder: {sub_folder}, Word Count: {folder_word_count}")
    print(f"Total Word Count for All Files: {total_word_count}")

# Specify the root folder containing your files
root_folder = '/content/drive/MyDrive/NLP_2/english'

count_words_in_folders(root_folder)


##### Results output ######

# Folder: history_de, Word Count: 10358
# Folder: disease_de, Word Count: 85191
# Folder: tech_de, Word Count: 23498
# Total Word Count for All Files: 119047

Folder: history_en, Word Count: 132870
Folder: disease_en, Word Count: 257739
Folder: tech_en, Word Count: 33588
Total Word Count for All Files: 424197


In [ ]:
!pip install spacy


In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [37]:
import spacy

# Loading English and German language models from spacy
nlp_english = spacy.load("en_core_web_sm")
nlp_german = spacy.load("de_core_news_sm")

# the `nlp.max_length` can be increased according to the size of corpus
nlp_english.max_length = 3000000
nlp_german.max_length = 3000000

# Function to process a text file and return a set of words, using spacy because it is pretrained on these language and know more words
def process_text_file(file_path, nlp_model):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        doc = nlp_model(text)
        words = {token.text for token in doc if token.is_alpha}
        return words

english_file_path = '/content/drive/MyDrive/NLP_2/combined_english.txt'
german_file_path = '/content/drive/MyDrive/NLP_2/combined_german.txt'

english_words = process_text_file(english_file_path, nlp_english)
german_words = process_text_file(german_file_path, nlp_german)

# For finding common terminology between English and German text files taking intersection of both files
common_terminology = english_words.intersection(german_words)





In [38]:
print(len(common_terminology))

1853


In [39]:
# Calculating shared terminology coverage
shared_terminology = english_words.intersection(german_words)
english_coverage = len(shared_terminology) / len(english_words) * 100
german_coverage = len(shared_terminology) / len(german_words) * 100

# Printing the shared terminology coverage
print("Shared Terminology Coverage:")
print(f"Shared Terminology Count: {len(shared_terminology)} words")
print(f"English Corpus Coverage: {english_coverage:.2f}%")
print(f"German Corpus Coverage: {german_coverage:.2f}%")


Shared Terminology Coverage:
Shared Terminology Count: 1853 words
English Corpus Coverage: 6.17%
German Corpus Coverage: 8.40%
